In [ ]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import yfinance as yf
from datetime import datetime
from sklearn.covariance import LedoitWolf


In [ ]:
TICKERS = [
    # Large-cap tech & communication
    "AAPL","MSFT","GOOGL","AMZN","TSLA","META","NVDA","NFLX","ADBE","CRM",

    # Semiconductors
    "INTC","AMD","QCOM","AVGO","TXN","MU","AMAT","LRCX","NXPI","KLAC",

    # Banks & financials
    "JPM","BAC","WFC","C","GS","MS","V","MA","PYPL","AXP",

    # Healthcare & pharma
    "PFE","JNJ","MRK","UNH","ABBV","LLY","BMY","GILD","AMGN","REGN",

    # Energy
    "XOM","CVX","BP","SHEL","TTE","COP","SLB","HAL","MPC","PSX",

    # Consumer staples & discretionary
    "PG","KO","PEP","WMT","COST","HD","MCD","NKE","TGT","SBUX",

    # Industrials
    "BA","CAT","DE","HON","GE","UPS","MMM","LMT","RTX","FDX",

    # Utilities
    "NEE","DUK","SO","AEP","EXC","SRE","WEC","PEG","ED","XEL",

    # Telecom
    "VZ","T","TMUS","CMCSA","CHTR","DIS","FOX","FOXA","ROKU",

    # More large-caps for diversity
    "ORCL","IBM","SAP","CSCO","MMM","BKNG","SPGI","BLK","MO","PM",
    "ADSK","WDAY"
]

In [ ]:
END = datetime.today().strftime("%Y-%m-%d")
RISK_FREE_RATE = 0.035
ANNUAL_TRADING_DAYS = 252
OUTPUT_DIR = "optiverse_outputs"

L2_REG = 1e-2
EPS_JITTER = 1e-5
ALLOW_SHORT = False
N_POINTS = 50
os.makedirs(OUTPUT_DIR, exist_ok=True)

LOOKBACK_YEARS = [3, 5, 10]
MIN_DATA_FRACTION = 0.9

In [ ]:
def download_prices(tickers, start, end=END):
    raw = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=False)
    if raw.empty:
        raise RuntimeError("yfinance returned no data for the given tickers / date range.")
    if isinstance(raw.columns, pd.MultiIndex):
        if "Adj Close" in raw.columns.get_level_values(0):
            df = raw["Adj Close"]
        elif "Close" in raw.columns.get_level_values(0):
            df = raw["Close"]
        else:
            df = raw.xs(raw.columns.levels[0][-1], axis=1, level=0, drop_level=True)
    else:
        if "Adj Close" in raw.columns:
            df = raw["Adj Close"]
        elif "Close" in raw.columns:
            df = raw["Close"]
        else:
            df = raw
    if isinstance(df, pd.Series):
        df = df.to_frame()
    df = df.dropna(how="all")
    if list(df.columns) != list(tickers) and len(df.columns) == len(tickers):
        try:
            df.columns = tickers
        except Exception:
            pass
    return df

#this function is used to download the prices from yfinance for the following stocks in tickers from the dates containing from start to end.
#and placing them properly in a dataframe.